In [1]:
import numpy as np
import pickle

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

from typing import Union, List, Tuple

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');

1.Znajdź listę wszystkich filmów o tej samej długości.

In [2]:
df = pd.read_sql("SELECT length,count(title) from film group by length",con = connection)
print(df)

     length  count
0       129      7
1       106      6
2       120      9
3       171      8
4       138      6
..      ...    ...
135     159      6
136      72      4
137     177      6
138     141      7
139     144      8

[140 rows x 2 columns]


2.Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [3]:
df2 = pd.read_sql("SELECT city.city,count(customer_id) \
                  from ((customer inner join address on customer.address_id = address.address_id)\
                  inner join city on address.city_id=city.city_id)\
                  group by city",con = connection)
print(df2)

                     city  count
0                 Asuncin      1
1              Rae Bareli      1
2               Molodetno      1
3                 Yerevan      1
4    Vitria de Santo Anto      1
..                    ...    ...
592                  Abha      1
593               Kolpino      1
594          Garden Grove      1
595          Novoterkassk      1
596              Maracabo      1

[597 rows x 2 columns]


3.Oblicz średni koszt wypożyczenia wszystkich filmów.

In [4]:
df3 = pd.read_sql("SELECT avg(amount) from payment",con=connection)
print(df3)

        avg
0  4.200606


4.Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [5]:
df4 = pd.read_sql("SELECT category.name,count(title) from ((film inner join film_category on film.film_id=film_category.film_id)\
inner join category on film_category.category_id = category.category_id)group by name",con=connection)
print(df4)

           name  count
0        Horror     56
1        Comedy     58
2        Sci-Fi     61
3         Drama     62
4       Foreign     73
5      Classics     57
6         Games     61
7           New     63
8        Travel     57
9         Music     51
10       Action     64
11  Documentary     68
12       Sports     74
13    Animation     66
14       Family     69
15     Children     60


5.Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [6]:
df5 = pd.read_sql("SELECT country.country,count(customer_id) \
                  from ((customer inner join address on customer.address_id = address.address_id)\
                  inner join city on address.city_id=city.city_id)\
                  inner join country on city.country_id=country.country_id\
                  group by country",con = connection)
print(df5)

                  country  count
0                Cambodia      2
1                  Turkey     15
2                 Germany      7
3              Madagascar      1
4                    Chad      1
..                    ...    ...
103                 Sudan      2
104  United Arab Emirates      3
105              Tanzania      3
106      French Polynesia      2
107              Bulgaria      2

[108 rows x 2 columns]


6.Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [7]:
df6 = pd.read_sql("Select store.address_id,store.store_id,count(customer_id)\
                  from customer inner join store on customer.store_id=store.store_id \
                  group by store.store_id having count(*) between 100 and 300",con = connection)
print(df6)

   address_id  store_id  count
0           2         2    273


7.Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [8]:
df7 = pd.read_sql("Select customer.first_name,customer.last_name,sum(film.length)\
                from customer inner join rental on customer.customer_id=rental.customer_id\
                inner join inventory on rental.inventory_id=inventory.inventory_id\
                inner join film on inventory.film_id=film.film_id group by customer.first_name,customer.last_name \
                having sum(film.length) > 1200",con=connection)
print(df7)

    first_name    last_name   sum
0        Scott      Shelley  2916
1          Tim         Cary  4476
2       Samuel       Marlow  2291
3         Vera        Mccoy  2275
4       Nelson  Christenson  2119
..         ...          ...   ...
594      Erica     Matthews  2727
595    Stanley    Scroggins  3694
596      Ellen      Simpson  2977
597      Patsy     Davidson  3513
598  Gwendolyn          May  2890

[599 rows x 3 columns]


8.Oblicz średnią wartość wypożyczenia filmu.

In [9]:
df8 = pd.read_sql("SELECT title,avg(payment.amount) from ((film inner join inventory on film.film_id=inventory.film_id)\
                        inner join rental on inventory.inventory_id=rental.inventory_id)\
                        inner join payment on rental.rental_id = payment.rental_id\
                        group by title",con=connection)
print(df8)

                           title       avg
0                 Frontier Cabin  5.990000
1    Arachnophobia Rollercoaster  4.772609
2             Cruelty Unforgiven  1.156667
3         Intolerable Intentions  5.704286
4                  Monsoon Cause  6.101111
..                           ...       ...
953                  Reef Salute  2.171818
954                   Bugsy Song  5.101111
955                Orient Closer  4.754706
956          Brotherhood Blanket  4.294348
957                Boulevard Mob  3.552500

[958 rows x 2 columns]


9.Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [10]:
df9 = pd.read_sql("SELECT category.name,avg(length) from ((film inner join film_category on film.film_id=film_category.film_id)\
inner join category on film_category.category_id = category.category_id)group by name",con=connection)
print(df9)

           name         avg
0        Horror  112.482143
1        Comedy  115.827586
2        Sci-Fi  108.196721
3         Drama  120.838710
4       Foreign  121.698630
5      Classics  111.666667
6         Games  127.836066
7           New  111.126984
8        Travel  113.315789
9         Music  113.647059
10       Action  111.609375
11  Documentary  108.750000
12       Sports  128.202703
13    Animation  111.015152
14       Family  114.782609
15     Children  109.800000


10.Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [11]:
df10 = pd.read_sql("SELECT title,name from film,category where (name,char_length(title)) in \
(SELECT category.name,max(char_length(title)) from ((film inner join film_category on film.film_id=film_category.film_id)\
inner join category on film_category.category_id = category.category_id) group by name) order by name desc",con=connection)
print(df10)

                         title       name
0      Extraordinary Conquerer     Travel
1      Entrapment Satisfaction     Travel
2      Trainspotting Strangers     Travel
3      Desperate Trainspotting     Travel
4    Satisfaction Confidential     Sports
..                         ...        ...
157     Deliverance Mulholland  Animation
158    Entrapment Satisfaction     Action
159    Trainspotting Strangers     Action
160    Desperate Trainspotting     Action
161    Extraordinary Conquerer     Action

[162 rows x 2 columns]


11.Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [12]:
df11 = pd.read_sql("SELECT title,name,length from film,category where (name,length) in \
(SELECT category.name,max(length) from ((film inner join film_category on film.film_id=film_category.film_id)\
inner join category on film_category.category_id = category.category_id) group by name) order by name desc",con=connection)
print(df11)

                  title    name  length
0        Darn Forrester  Travel     185
1    Soldiers Evolution  Travel     185
2           Gangs Pride  Travel     185
3          Worst Banger  Travel     185
4         Muscle Bright  Travel     185
..                  ...     ...     ...
137       Chicago North  Action     185
138  Soldiers Evolution  Action     185
139       Muscle Bright  Action     185
140           Home Pity  Action     185
141   Sweet Brotherhood  Action     185

[142 rows x 3 columns]
